In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [2]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation

from imfdtd.fdtd import FDTD
from imfdtd.consts import c0

In [3]:
from imfdtd.utils import *
from imfdtd.object import GeoObject

from matplotlib.patches import Ellipse, PathPatch

In [4]:
%matplotlib tk

In [5]:
CM = 1E-2
r1= 5*CM
r2 = 10*CM

In [6]:
fdtdsolver = FDTD([[-3*r2, 3*r2], [-3*r2, 3*r2]], [300, 300])

In [7]:
Epoints, Hxpoints, Hypoints = FDTD.generate_yee_grid(
    [[-4*r2, 4*r2], [-3*r2, 3*r2]], [400, 400]
    )

dx = Epoints[0][0][1]-Epoints[0][0][0]
dt = dx/(2*c0) 

E_field = np.zeros((*Epoints[0].shape ,3))
H_field_x = np.zeros((*Hxpoints[0].shape,))
H_field_y = np.zeros((*Hypoints[0].shape,))

X, Y = Epoints

In [8]:
I3 = np.eye(3)
rel_e_ten = np.zeros((*X.shape, 3, 3))
rel_e_ten[..., : , :] = I3

In [9]:
# Get relative tensor
s_xyz, s_cylinder, s_sph = get_basic_coordinates()
r_s, phi_s, z_s = s_cylinder
R1, R2 = sp.symbols("R_1, R_2")

# Define transoformation
u1 = R1 + ((R2- R1)/R1)*r_s
u2 = phi_s
u3 = z_s
eq_coor = [u1.subs([(R1, r1),(R2, r2)]), u2, u3]
syms = [r_s, phi_s, z_s]

In [10]:
gt = get_invisible_tensor(eq_coor, syms, s_xyz)
g_ij = get_func_gij(s_xyz[:2], gt)

In [11]:
e_out = Ellipse([0,0], width = r2, height=r2, angle =0)
e_in = Ellipse([0,0], width = r1, height=r1, angle =0)

points = np.vstack([X.ravel(), Y.ravel()]).T

In [12]:
# Get inner lattice
p_index_out = e_out.contains_points(points) #
p_index_in = e_in.contains_points(points)
p_index = np.logical_xor(p_index_out, p_index_in) 

index = np.where(p_index)[0]

nx, ny = X.shape
row_i, col_i = (index/nx).astype(int), index%nx

X_rel = X[row_i, col_i]
Y_rel = Y[row_i, col_i]

# Set rel_tensor value
rel_e_ten[row_i, col_i,:,:] = g_ij(X_rel, Y_rel)

In [13]:
rel_e = rel_e_ten[:,:, 2, 2]

In [14]:
rel_e[:,:150] = 1
rel_e[:, 250:] = 1

In [15]:
rel_e.min()

0.024993639306157365

In [16]:
#plt.imshow(rel_e)

In [17]:
#E_field[:, :, 2] = np.exp(-((X)**2+Y**2)/0.2) 
E_field[:, :, 2] = 5*np.exp(-((X)**2 + (Y)**2)/0.01) 

In [18]:
def curlE(E):
    ny, nx = E.shape[:2]
    curl_hx = np.zeros((ny+1, nx))
    curl_hy = np.zeros((ny, nx+1))

    curl_hx[1:-1,:] = curl_hx[1:-1,:] + np.diff(E[:,:,2], axis=0)
    curl_hy[:,1:-1] = curl_hy[:,1:-1] - np.diff(E[:,:,2], axis=1)
    return curl_hx, curl_hy
def curlH(Hx, Hy):
    return np.diff(Hy, axis=1)- np.diff(Hx, axis=0)

def mv_mul(
        matrix_arr:np.ndarray, 
        vector_arr:np.ndarray):
    # Ensure that the shapes of the input arrays are compatible
    assert matrix_arr.shape[:2] == vector_arr.shape[:2], "Shapes of matrix and vector arrays must be the same (N, M)."
    assert matrix_arr.shape[-2:] == (3, 3), "Matrix array must have shape (N, M, 3, 3)."
    assert vector_arr.shape[-1] == 3, "Vector array must have shape (N, M, 3)."

    # Perform element-wise matrix-vector multiplication
    result = np.einsum('ijkl,ijl->ijk', matrix_arr, vector_arr)

    return result

In [19]:
#%matplotlib tk 

In [20]:
curl_hx, curl_hy = curlE(E_field)
H_field_x -= 0.5*curl_hx
H_field_y -= 0.5*curl_hy
E_field[:,:, 2] = 0.5*curlH(H_field_x, H_field_y)

In [24]:
E_field = np.zeros((*Epoints[0].shape ,3))
D_field = np.zeros((*Epoints[0].shape ,3))
H_field_x = np.zeros((*Hxpoints[0].shape,))
H_field_y = np.zeros((*Hypoints[0].shape,))

In [22]:
def func_animate(frame_index, E_field, H_field_x, H_field_y, e_out, e_in):
    #print(frame_index)
    curl_hx, curl_hy = curlE(E_field)
    H_field_x -= 0.5*curl_hx
    H_field_y -= 0.5*curl_hy
    D_field[:,:, 2] += 0.5*curlH(H_field_x, H_field_y)
    E_field[:,:, 2] = rel_e*D_field[:,:, 2] 

    # Source
    p_index =  (X<-3.3*r2)
    E_field[p_index, 2] = np.sin(5E10*dt*frame_index)
    #p_index = ((X+r2)**2 + (Y)**2)<0.00005

    #E_field[p_index, 2] = 5*np.exp(-((X+r2)**2 + (Y)**2)/0.05)[p_index]*np.sin(5E10*dt*frame_index)

    image.set_data(E_field[:, :, 2])
    ax.set_title(str(frame_index))
    #ax.add_patch(e_out)
    #ax.add_patch(e_in)
    #print(frame_index)
    return image, ax

In [23]:
fig = plt.figure()
ax = fig.gca()

im = np.random.randn(100,100)
image = ax.imshow(E_field[:,:,2], animated=True, cmap="magma", vmin=-1., vmax=1.)

anim = animation.FuncAnimation(fig, 
    func_animate, 
    fargs = [E_field, H_field_x, H_field_y, e_out, e_in],
    interval=10, frames=300, blit=True)
plt.show()

---